In [28]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [9]:
df1 = pd.read_parquet("./metrics/raw/train-00000-of-00004-49a07627b3b5bdbe.parquet", engine="pyarrow")
df2 = pd.read_parquet("./metrics/raw/train-00001-of-00004-62d2ea1ccbf3c546.parquet", engine="pyarrow")
df3 = pd.read_parquet("./metrics/raw/train-00002-of-00004-5d4368eca33ee435.parquet", engine="pyarrow")
df4 = pd.read_parquet("./metrics/raw/train-00003-of-00004-ef0356d35c1172f0.parquet", engine="pyarrow")
df = pd.concat([df1, df2, df3, df4])
df

,prompt,response,source
0,<p>Good morning</p>\n\n<p>I have a Wpf datagri...,One possible solution is to use a fixed width ...,pacovaldez/stackoverflow-questions
1,"<h2>Hi, How can I generate a pdf with the scre...","To generate a PDF with the screen visual data,...",pacovaldez/stackoverflow-questions
2,<pre><code>package com.kovair.omnibus.adapter....,The issue might be related to class loading an...,pacovaldez/stackoverflow-questions
3,<p>I'm trying to get it so that all of the ite...,"To make the whole page scrollable, remove the ...",pacovaldez/stackoverflow-questions
4,<p>I have used a <code>ListView</code> and the...,The issue seems to be with the layout paramete...,pacovaldez/stackoverflow-questions
...,...,...,...
202198,Who is the founder of adidas?,Adi Dassler,unified_chip2
202199,who plays jim halpert?,John Krasinski.,unified_unifiedskg_instructions
202200,who won the 1998 world cup,France.,unified_hc3_human
202201,who sang cool jerk,The Capitols.,unified_hc3_human


In [28]:
def processPrompt(prompt):
    if ("<" in prompt):
        return 
    parsed = BeautifulSoup(prompt)
    x = [t.getText() for t in parsed.findAll("p")]
    s = ""
    for i in x:
        s += i
    return s

In [30]:
df["prompt"] = df["prompt"].apply(processPrompt)

/var/folders/wt/0zq9sp6n1n9bl5h1gw4l2kph0000gn/T/ipykernel_7675/3449371.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  parsed = BeautifulSoup(prompt)
/var/folders/wt/0zq9sp6n1n9bl5h1gw4l2kph0000gn/T/ipykernel_7675/3449371.py:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  parsed = BeautifulSoup(prompt)


In [31]:
df["prompt"]

0         Good morningI have a Wpf datagrid that is disp...
1         I would like to generate a pdf with the data t...
2         In the above code snippet I am loading a jar f...
3         I'm trying to get it so that all of the items ...
4         I have used a ListView and the parent in the x...
                                ...                        
202198                        Who is the founder of adidas?
202199                               who plays jim halpert?
202200                           who won the 1998 world cup
202201                                   who sang cool jerk
202202                                     what state is Mn
Name: prompt, Length: 808812, dtype: object

In [13]:
df.to_csv("./metrics/parsed/out.txt")

In [14]:
pd.read_csv("./metrics/parsed/out.txt")

,Unnamed: 0,prompt,response,source
0,0,<p>Good morning</p>\n\n<p>I have a Wpf datagri...,One possible solution is to use a fixed width ...,pacovaldez/stackoverflow-questions
1,1,"<h2>Hi, How can I generate a pdf with the scre...","To generate a PDF with the screen visual data,...",pacovaldez/stackoverflow-questions
2,2,<pre><code>package com.kovair.omnibus.adapter....,The issue might be related to class loading an...,pacovaldez/stackoverflow-questions
3,3,<p>I'm trying to get it so that all of the ite...,"To make the whole page scrollable, remove the ...",pacovaldez/stackoverflow-questions
4,4,<p>I have used a <code>ListView</code> and the...,The issue seems to be with the layout paramete...,pacovaldez/stackoverflow-questions
...,...,...,...,...
808807,202198,Who is the founder of adidas?,Adi Dassler,unified_chip2
808808,202199,who plays jim halpert?,John Krasinski.,unified_unifiedskg_instructions
808809,202200,who won the 1998 world cup,France.,unified_hc3_human
808810,202201,who sang cool jerk,The Capitols.,unified_hc3_human


In [40]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from scipy import sparse

In [62]:
transformer = SentenceTransformer("all-MiniLM-L6-v2", device="mps")
df

,prompt,response,source
0,<p>Good morning</p>\n\n<p>I have a Wpf datagri...,One possible solution is to use a fixed width ...,pacovaldez/stackoverflow-questions
1,"<h2>Hi, How can I generate a pdf with the scre...","To generate a PDF with the screen visual data,...",pacovaldez/stackoverflow-questions
2,<pre><code>package com.kovair.omnibus.adapter....,The issue might be related to class loading an...,pacovaldez/stackoverflow-questions
3,<p>I'm trying to get it so that all of the ite...,"To make the whole page scrollable, remove the ...",pacovaldez/stackoverflow-questions
4,<p>I have used a <code>ListView</code> and the...,The issue seems to be with the layout paramete...,pacovaldez/stackoverflow-questions
...,...,...,...
202198,Who is the founder of adidas?,Adi Dassler,unified_chip2
202199,who plays jim halpert?,John Krasinski.,unified_unifiedskg_instructions
202200,who won the 1998 world cup,France.,unified_hc3_human
202201,who sang cool jerk,The Capitols.,unified_hc3_human


In [107]:
out = df.head(10000)["prompt"].apply(transformer.encode)
# out = [transformer.encode("Who is the founder of adidas?")]

In [106]:
out = np.array([np.array(i) for i in out]) * (10**6)
iso_forest = IsolationForest(contamination=0.5).fit(out)
y = [transformer.encode(")(*&^%)") * (10**6)]
print(iso_forest.predict(y))

[1]


In [94]:
out * (10**6)

array([[ -28775.28592944,   23216.73184633,  -31210.72985232, ...,
         -19098.00805151, -140103.02722454,   50745.6921041 ],
       [ -35899.18091893,  -17474.97543693,  -89191.25795364, ...,
          12688.05004656,   34010.71950793,   64563.63946199],
       [  40787.38391399,   10782.295838  , -108611.60606146, ...,
         -18576.29790902,  -19721.22862935,   38031.488657  ],
       ...,
       [   8862.04373091,  -24976.44163668,   61334.29706097, ...,
           2759.10948403,   30729.91222143,   31420.85298896],
       [  -8359.78426039,   17789.79040682,   91523.04381132, ...,
        -104984.55166817,    1941.63352717,   28838.33274245],
       [  -4270.86744457,  -14009.80167091,  -16009.19105113, ...,
        -115915.99136591,   15819.70229745,   10307.1751073 ]])